# Content Loss for Super Resolution
- L1, L2 Loss
- Perceptual Loss
- Texture/Style Loss 

In [3]:
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.models import Model
from keras.layers import Input
import keras.backend as K
import tensorflow as tf
import cv2

Using TensorFlow backend.


### 1. L1, L2(MSE) Loss

$L_1=\parallel I_{SR}-I_{HR} \parallel$

$L_2=\parallel I_{SR}-I_{HR} \parallel_2^2$

Due to using full image information, those losses will produce over-smoothed images that lack high-frequency textures and do not look natural despite yielding high PSNR values.

In [3]:
def l1_loss(y_true, y_pred):
    return K.mean(K.abs(y_true - y_pred))

def l2_loss(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred))

### 2. Perceptual Loss

$L_{P}=\parallel \phi(I_{SR})-\phi(I_{HR}) \parallel_2^2$

$\phi(\cdot)$ is one layer's output of VGG19.

The result of perceptual loss looks slightly sharper than MSE, but it also produces unpleasing checkerboard artifacts.

In [4]:
def perceptual_loss(y_true, y_pred):  # y_true and y_pred's pixels are scaled between 0 to 255
    y_true = preprocess_input(y_true)
    y_pred = preprocess_input(y_pred)
    vgg = VGG19(include_top=False, weights='imagenet', input_shape=(256,256,3))
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true)-loss_model(y_pred)))

### 3. Texture/Style Loss

$L_{T}=\parallel G(\phi(I_{SR}))-G(\phi(I_{HR})) \parallel_2^2$

Gram matrix $G(F) = F^TF$

It helps to generate more realistic textures.

In [2]:
def gram_matrix(x):
    b, h, w, c = x.shape
    size = h * w * c
    feature = K.reshape(x, (b, h * w, c))
    return tf.matmul(feature, feature, transpose_a=True) / size

def style_loss(y_true, y_pred):
    y_true = preprocess_input(y_true)
    y_pred = preprocess_input(y_pred)
    vgg = VGG19(include_top=False, weights='imagenet', input_shape=(256,256,3))
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(gram_matrix(loss_model(y_true))-gram_matrix(loss_model(y_pred))))